In [6]:
import gym
import numpy as np
import random
from utils.TileCoding import IHT, tiles

env = gym.make('MountainCar-v0')
maxtiles = 2048


numtilings = 8

max_position, max_velocity = env.observation_space.high
min_position, min_velocity = env.observation_space.low

position_scale = numtilings / (max_position - min_position)
velocity_scale = numtilings / (max_velocity - min_velocity)

hash_table = IHT(maxtiles)


def get_active_tiles(position, velocity, action):
    active_tiles = tiles(hash_table, numtilings,
                         [position_scale * position, velocity_scale * velocity],
                         [action])
    return active_tiles

def features(observation, action):
    active_tiles = get_active_tiles(observation[0], observation[1], action)
    feature = [0.] * maxtiles
    for idx in active_tiles:
        feature[idx] = 1
    return feature

[2017-06-12 14:15:12,682] Making new env: MountainCar-v0


In [7]:
def q_approx(o, a, w):
    f = features(o, a)
    s = 0.
    for i in range(maxtiles):
        s += f[i] * w[i]
    return s

def eps_greedy(env, o, eps, w):
    if random.random() <= 1 - eps:
        return np.argmax([q_approx(o, a, w) for a in range(env.action_space.n)])
    else:
        return random.randint(0, 2)

In [8]:
def semi_gradient_sarsa(env, alpha, gamma, eps, iter_cnt=5000, show=20):
    w = np.array([0.] * maxtiles)

    i = 0
    while i < iter_cnt:
        o = env.reset()
        R = 0
        a = eps_greedy(env, o, eps, w)
        
        ss = (i % show == 0)
        
        while True:
            oo, r, done, info = env.step(a)
            if ss:
                env.render()
            
            R += r
            
            if done:
                w += (alpha * (r - q_approx(o, a, w))) * np.array(features(o, a))
                break
            
            aa = eps_greedy(env, oo, eps, w)
            q_n = q_approx(oo, aa, w)
            q = q_approx(o, a, w)
            w += (alpha * (r + gamma * q_n - q)) * np.array(features(o, a))
            o, a = oo, aa
        
        i += 1


In [9]:
semi_gradient_sarsa(env, alpha=0.01, gamma=1., eps=0.2)

env.close()

ArgumentError: argument 2: <class 'TypeError'>: wrong type